In [ ]:
import os
import pandas as pd

In [ ]:
text = """
Gli abitanti di Little Hangleton la chiamavano ancora Casa Riddle, anche se erano passati tanti anni da quando i Riddle ci abitavano. 
Si trovava sulla collina che dominava il villaggio: alcune delle finestre erano inchiodate, al tetto mancavano delle tegole e 
l'edera cresceva incolta sulla facciata. Un tempo Casa Riddle era stata una dimora elegante, certo l'edificio più vasto e grandioso 
nel raggio di chilometri, ma ora era umida, desolata e disabitata. Gli hangletoniani convenivano tutti che la vecchia casa era 
sinistra. Mezzo secolo prima, qualcosa di strano e terribile era successo là dentro, qualcosa di cui gli abitanti più anziani del 
villaggio amavano ancora discutere quando erano a corto di pettegolezzi. La storia era stata ripetuta così tante volte, e vi erano 
stati aggiunti così tanti fronzoli che nessuno era più certo di quale fosse la verità. Ogni versione del racconto, comunque, 
cominciava allo stesso modo: cinquant'anni prima, all'alba di una bella giornata d'estate, quando Casa Riddle era ancora ben tenuta e 
imponente, una cameriera era entrata in salotto e aveva trovato morti tutti e tre i Riddle. La cameriera era corsa urlando giù per la 
collina fino al villaggio e aveva radunato tutte le persone che poteva. Sono là stesi con gli occhi spalancati! Freddi come il 
ghiaccio! Ancora vestiti per la cena! Fu chiamata la polizia e tutta quanta Little Hangleton si crogiolò in una curiosità atterrita e 
in una malcelata eccitazione. Nessuno si sforzò di fingersi addolorato per i Riddle, che erano stati assolutamente impopolari. Gli 
anziani signori Riddle, marito e moglie, erano ricchi, snob e sgarbati, e loro figlio Tom, ormai adulto, era anche peggio. Tutto 
quello che importava agli abitanti era l'identità dell'assassino: chiaramente, tre persone in apparenza sane non morivano di colpo per 
cause naturali nella stessa notte. L'Impiccato, il pub locale, fece affari d'oro quella sera: il villaggio al completo accorse per 
discutere gli omicidi. E la ricompensa per quell'uscita serale arrivò quando la cuoca dei Riddle fece un ingresso teatrale e annunciò 
al pub improvvisamente silenzioso che un uomo chiamato Frank Bryce era stato appena arrestato.
"""

In [ ]:
def get_all_csv_files_rec(dir):
    """Ritorna una lista di tutti i file .csv nella directory e nelle sottodirectory."""
    csv_files = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files
all_csv_files = get_all_csv_files_rec('dataframes')

In [ ]:
all_df = pd.DataFrame()
for path in all_csv_files:
    assert(os.path.exists(path))
    # print(path)
    df = pd.read_csv(path)
    all_df = pd.concat([all_df, df], ignore_index=True)
len(all_df), all_df.columns

In [ ]:
all_df.drop_duplicates(['term_italian', 'base_lemma_italian', 'base_lemma_english', 'part_of_speech'], inplace=True)
len(all_df)

In [ ]:
all_df['base'] = all_df.apply(lambda row: f"{row['base_lemma_italian']} - {row['base_lemma_english']}", axis=1)
all_df['base'].value_counts()

In [ ]:
all_df['base'].value_counts()[-10:]
for base in all_df['base'].value_counts()[-10:].index:
    # print base and part_of_speech
    base_df = all_df[all_df['base'] == base]
    print(f"Base: {base}, Part of speech: {base_df['part_of_speech'].iloc[0]}")

In [ ]:
all_df[all_df['base'] == 'ognuno - everyone (feminine)']

In [ ]:
ignore_words = [
    'bryce',
    "cinquant'anni",
    'frank',
    'hangleton',
    'little',
    'riddle',
    'tom',
]
harry_potter_words = set(text.lower().replace('\n', ' ').replace(':', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace('?', ' ').split(' '))
# ignore empty strings
harry_potter_words.discard('')
# ignore numbers
harry_potter_words = {w for w in harry_potter_words if not w.isdigit()}
harry_potter_words = harry_potter_words - set(ignore_words)
len(harry_potter_words)

In [ ]:
IMPORTANT_COLS = ['term_italian', 'part_of_speech', 'translation_english', 'base_lemma_italian', 'base_lemma_english']

In [ ]:
all_df[all_df['term_italian'].isin(harry_potter_words)][IMPORTANT_COLS]

In [ ]:
missing_words = harry_potter_words - set(all_df['term_italian'])
print(len(missing_words))
missing_words

In [ ]:
# The residents of Little Hangleton still called it Riddle House, even though it had been many years since the Riddles had lived there.
sentence_1 = "Gli abitanti di Little Hangleton la chiamavano ancora Casa Riddle, anche se erano passati tanti anni da quando i Riddle ci abitavano."

# It stood on the hill overlooking the village: some of the windows were nailed shut, tiles were missing from the roof, and ivy was growing wild on the facade.
sentence_2 = "Si trovava sulla collina che dominava il villaggio: alcune delle finestre erano inchiodate, al tetto mancavano delle tegole e l'edera cresceva incolta sulla facciata."

# Riddle House had once been an elegant mansion, certainly the largest and grandest building for miles around, but now it was damp, desolate, and uninhabited.
sentence_3 = "Un tempo Casa Riddle era stata una dimora elegante, certo l'edificio più vasto e grandioso nel raggio di chilometri, ma ora era umida, desolata e disabitata."

# Hangletonians all agreed that the old house was sinister.
sentence_4 = "Gli hangletoniani convenivano tutti che la vecchia casa era sinistra."

sentence = sentence_1

In [ ]:
sentence_italian_words = list(sentence.lower().replace('\n', ' ').replace(':', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace('?', ' ').split(' '))
sentence_italian_words = [w for w in sentence_italian_words if w != '']
sentence_italian_words

In [ ]:
all_df.columns

In [ ]:
from fuzzy_match import find_translation

In [ ]:
all_df[all_df['term_italian'] == 'gli']

In [ ]:
# find_translation(all_df, 'crogiolò', min_similarity=0.8)
# find_translation(all_df, 'little', min_similarity=0.9)

In [ ]:
def get_trans(word_italian):
    ret = find_translation(all_df, word_italian, min_similarity=0.8)
    if ret is None:
        return word_italian, None, None
    it, en = ret
    return word_italian, it, en
sentence_english_trans = [
    get_trans(word) for word in sentence_italian_words
]
new_sentence = ''
for original_word, match_it, match_en in sentence_english_trans:
    print(f"{original_word} -> {match_it} -> {match_en}")
    new_sentence += f"{original_word} [{match_en}] "
new_sentence

In [ ]:
sentence_italian_words

In [ ]:
i = 7
it_word = sentence_italian_words[i]
print(f"{it_word} - {find_translation(all_df, it_word, min_similarity=0.8)}")